# Introduction 
In aerospace, we use plot3D files to represent the fluid domain around a geometry. This allows us to perform simulations and calculate flow physics inside of each cell/rectangle. Often the connectivity of these blocks are not easily obtainable. You would have to buy an expensive tool that will create the mesh and output some kind of connectivity file. Sometimes these files are proprietary which doesn't help research. This `plot3d` library in python can be used to find connectivity information and also split the blocks into smaller blocks for evaluation. 

**The goal with this project is to enable research and learning. You shouldn't need to purchase something expensive to calculate all this for you.**


## Plot of Turbine Stage (Stator + Rotor Blocks)
Below shows a plot in paraview of the `StageMesh.xyz` file. Each color represents a rectangular block with 6 sides. I've circled the blocks related to the stator and the blocks related to the rotor. It's important to plot this in paraview because this allows you to split the blocks into `Stator` blocks and `Rotor` blocks later. 

> Note: I am using paraview 5.10.1 to plot. Different versions of paraview may have differences in the way python macros are coded so you may see errors.

![stage_mesh](stagemesh.png)


## Objectives of this Notebook
1. Split the blocks and find connectivity
2. Identify surfaces that should be marked as solid
3. Identify the mixing plane location 

![mixing plane](stage_mixing_plane.png)

In [ ]:
# Imports
import sys
from typing import List
from plot3d import read_plot3D, connectivity_fast, rotated_periodicity, write_plot3D, Direction, split_blocks
import os, pickle
import numpy as np
import json 

## Sanity Check Finding negative volumes. 
This should be done before doing anything else. A negative volume will certainly crash the solver.

In [ ]:
mesh_filename = 'StageMesh.xyz'
print("Reading mesh")
blocks = read_plot3D(mesh_filename,binary=True,big_endian=False)

# This part checks for negative volumes 
lines = list() 
negative_volumes = False

for i in range(len(blocks)):
    v = blocks[i].cell_volumes()
    if np.min(v)<0:
        negative_volumes = True
        lines.append(f'negative volume found in block number: {i} \n')
        print(lines[-1])

if negative_volumes:
    with open('negative_volume.txt', 'w') as f:
        for line in lines:
            f.write(line)         

## Creating the mesh for stator and rotor
In this step, what I did was plot the entire geometry in paraview then deactivated the blocks to figure out which blocks belonged to the stator and rotor.
Blocks 0 to 3 are for the stator and the rest are the rotor. The code below splits the blocks and exports stator and rotor separately. 

In [ ]:
stator_blocks = [blocks[i] for i in range(0,4)]
rotor_blocks = [blocks[i] for i in range(4,len(blocks))]
write_plot3D('stator.xyz',stator_blocks,binary=True)
write_plot3D('rotor.xyz',rotor_blocks,binary=True)
stator_blocks_split = split_blocks(stator_blocks,500000, direction=Direction.i) # Splits the blocks while keeping the gcd. 380,000 is a rough number that it tries to match
rotor_blocks_split = split_blocks(rotor_blocks,700000, direction=Direction.i)
write_plot3D('stator_split.xyz',stator_blocks_split,binary=True)
write_plot3D('rotor_split.xyz',rotor_blocks_split,binary=True)

## Step 1: Split the blocks and find the connectivity
Splitting the blocks into smaller blocks can help the solver run much faster. To improve the time it takes to split and find connectivity/periodicity. You need to ensure each block of your original mesh is divisible by at least 4. The higher the greatest common divisor (gcd), the faster you get the connectivity information 

In [ ]:
def find_connectivity(filename:str,nblades:int):
    blocks = read_plot3D(f'{filename}.xyz',binary=True,big_endian=False)
    # Finds the connectivity 
    if not os.path.exists(f'{filename}_connectivity.pickle'):
        print('checking connectivity')
        face_matches, outer_faces_formatted = connectivity_fast(blocks)
        with open(f'{filename}_connectivity.pickle','wb') as f:
            [m.pop('match',None) for m in face_matches] # Remove the dataframe
            pickle.dump({"face_matches":face_matches, "outer_faces":outer_faces_formatted},f)


    # Finds the periodicity once connectivity is found   
    with open(f'{filename}_connectivity.pickle','rb') as f:
        data = pickle.load(f)
        face_matches = data['face_matches']
        outer_faces = data['outer_faces']
    
    print("Find periodicity")
    rotation_angle = 360.0/nblades 
    periodic_surfaces, outer_faces_to_keep,periodic_faces,outer_faces = rotated_periodicity(blocks,face_matches,outer_faces,rotation_axis='x',rotation_angle=rotation_angle)

    with open(f'{filename}_connectivity_periodicity.pickle','wb') as f:
        [m.pop('match',None) for m in face_matches] # Remove the dataframe
        pickle.dump({
            "face_matches":face_matches,
            "periodic_faces":periodic_surfaces,
            "outer_faces":outer_faces_to_keep       
            },f)

In [ ]:
find_connectivity(filename="stator_split",nblades=55)
print('stator done')
find_connectivity(filename="rotor_split",nblades=60)
print('rotor done')

## Plotting the Connectivity Information in Paraview
This tutorial was tested using Paraview 5.10.1. Different versions of paraview may not work. They keep changing things in the pvpython library that breaks compatibility with earlier versions. If there's a major update to paraview that you require support for for example version 6 or 5.20.1 then file a github issue and I'll take a look at it; but if it's 5.11.1 then no, because it's a minor update. Also feel free to submit fixes to pv_library.py for newer versions of paraview. 

There's 2 files that you need for plotting with paraview `pv_library.py` and `pv_stator_plot.py`. You will have to call paraview executable from within this directory using command prompt(windows)

[![Watch the video](https://img.youtube.com/vi/bgSSJQolR0E/default.jpg)](https://www.youtube.com/watch?v=bgSSJQolR0E)

1. Open command prompt in the directory containing `pv_stator_plot.py` and the `.xyz files`
2. Find the path to paraview. For me it's this path: `"C:\Program Files\ParaView 5.10.1-Windows-Python3.9-msvc2017-AMD64\bin\paraview.exe" --script=pv_stator_plot.py`
3. Hit [Enter] and this should load paraview and plot the connectivities from the pickle file. 

You can view the contents of `pv_stator_plot.py` to see how the connectivity information is structured. 

### What to look for
If you see `outer_face` plotted in Paraview, this indicates that it's an unmatched face - it can be either a shroud, hub, blade body, inlet, mixing-plane. **We want to identify each of those which is what the next section is about.**

![stator paraview](stator_paraview.png)

## Step 2: Identifying key surfaces - Stator 
Now that the data is in paraview, we need to select the surfaces and identify which ones are the inlet, hub, shroud, blade, and mixing plane. The way we do this is to use paraview to identify the block and the (IMIN, JMIN, KMIN, IMAX, JMAX, KMAX). Then we use `find_connected_face` functionality to identify faces that are connected to it by a common edge.

There's also some parts near the trailing edge that should be identify as periodic but are missed by `rotated_periodicity`. The miss could be attributed to points not lining up perfectly within 1E-6 tolerance. This will be investigated in a later update. 

Below is an example of how to do this for the stator. A separate example is provided for the rotor

In [ ]:
import sys
from plot3d import Face, find_connected_face,find_face,read_plot3D
import pickle
import numpy as np 
# Stator 
blocks = read_plot3D('stator_split.xyz',binary=True)
with open('stator_split_connectivity_periodicity.pickle','rb') as f:
    data = pickle.load(f)
    face_matches = data['face_matches']
    outer_faces = data['outer_faces']

# Stator body
block_id = 12; indices = np.array([0,0,0,68,148,0], dtype=int) # this is the face we need to find matches for
stator_face_to_match,_ = find_face(blocks,block_id, indices,outer_faces)
stator_faces,outer_faces = find_connected_face(blocks,stator_face_to_match, outer_faces)
stator_faces.append(stator_face_to_match.to_dict())

# Stator Hub
block_id = 0; indices = np.array([0,0,0,44,0,76],dtype=int)
stator_hub_to_match,_ = find_face(blocks,block_id, indices,outer_faces)
stator_hub, outer_faces = find_connected_face(blocks,stator_hub_to_match, outer_faces)
stator_hub.append(stator_hub_to_match.to_dict())

block_id = 6; indices = np.array([0,0,0,60,0,36],dtype=int) 
stator_hub_to_match,_ = find_face(blocks,block_id, indices,outer_faces)
stator_hub2, outer_faces = find_connected_face(blocks,stator_hub_to_match, outer_faces)
stator_hub2.append(stator_hub_to_match.to_dict())
stator_hub.extend(stator_hub2)

block_id = 9; indices = np.array([0,0,0,68,0,48],dtype=int) 
stator_hub_to_match,_ = find_face(blocks,block_id, indices,outer_faces)
stator_hub3, outer_faces = find_connected_face(blocks,stator_hub_to_match, outer_faces)
stator_hub3.append(stator_hub_to_match.to_dict())
stator_hub.extend(stator_hub3)

# Stator Shroud
block_id = 0; indices = np.array([0,148,0,44,148,76],dtype=int)
stator_shroud_to_match,_ = find_face(blocks,block_id, indices,outer_faces)
stator_shroud, outer_faces = find_connected_face(blocks,stator_shroud_to_match, outer_faces)
stator_shroud.append(stator_shroud_to_match.to_dict())

block_id = 6; indices = np.array([0,148,0,60,148,36],dtype=int) 
stator_shroud_to_match,_ = find_face(blocks,block_id, indices,outer_faces)
stator_shroud2, outer_faces = find_connected_face(blocks,stator_shroud_to_match, outer_faces)
stator_shroud2.append(stator_shroud_to_match.to_dict())
stator_shroud.extend(stator_shroud2)

block_id = 9; indices = np.array([0,148,0,68,148,48],dtype=int) 
stator_shroud_to_match,_ = find_face(blocks,block_id, indices,outer_faces)
stator_shroud3, outer_faces = find_connected_face(blocks,stator_shroud_to_match, outer_faces)
stator_shroud3.append(stator_shroud_to_match.to_dict())
stator_shroud.extend(stator_shroud3)

# Mixing Plane
block_id = 5; indices = np.array([36,0,0,36,148,76], dtype=int)
mixing_plane_face_to_match,_ = find_face(blocks,block_id, indices,outer_faces)
stator_mixing_plane_faces, outer_faces = find_connected_face(blocks,mixing_plane_face_to_match, outer_faces)
stator_mixing_plane_faces.append(mixing_plane_face_to_match.to_dict())

# Inlet
block_id = 0; indices = np.array([0,0,0,0,148,76], dtype=int)
inlet_face_to_match,_ = find_face(blocks,block_id, indices,outer_faces)
stator_inlet_faces, outer_faces = find_connected_face(blocks,inlet_face_to_match, outer_faces)
stator_inlet_faces.append(inlet_face_to_match.to_dict())

data['outer_faces'] = outer_faces
data['mixing_plane'] = stator_mixing_plane_faces
data['stator_body'] = stator_faces
data['stator_shroud'] = stator_shroud
data['stator_hub'] = stator_hub
data['inlet'] = stator_inlet_faces

# This is useful for viewing the mesh
with open('stator_split_connectivity_final.pickle','wb') as f:
    pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)
    print("done")


# Part of blade TE (Periodic)
face_match_1,indx = find_face(blocks,7, np.array([0,0,0,0,148,36], dtype=int),outer_faces)
outer_faces.pop(indx)
face_match_2,indx = find_face(blocks,8, np.array([0,0,48,36,148,48], dtype=int),outer_faces)
outer_faces.pop(indx)

data['face_matches'].append({'block1':face_match_1.to_dict(),
                            'block2':face_match_2.to_dict()})
data['outer_faces'] = outer_faces

with open('stator_split_connectivity_final.pickle','wb') as f:
    pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)
    print("done")


## Step 3: Identifying key surfaces - Rotor 
Similar to the stator we need to identify the faces corresponding to the hub, shroud, blade surface, mixing plane, and outlet for the rotor. 

Like the stator, there's also some parts near the trailing edge that should be identify as periodic but are missed by `rotated_periodicity`.  

In [ ]:
import sys
from plot3d import Face, find_connected_face,find_face,read_plot3D
import pickle
import numpy as np 

# Rotor
blocks = read_plot3D('rotor_split.xyz',binary=True)
with open('rotor_split_connectivity_periodicity.pickle','rb') as f:
    data = pickle.load(f)
    face_matches = data['face_matches']
    outer_faces = data['outer_faces']

# Mixing Plane
block_id = 0; indices = np.array([0,0,0,0,148,76], dtype=int) # this is the face we need to find matches for
rotor_mixing_plane_face_to_match,indx = find_face(blocks,block_id, indices,outer_faces)
rotor_mixing_plane_faces, outer_faces = find_connected_face(blocks,rotor_mixing_plane_face_to_match, outer_faces)# Should match with block 10 [0,0,0,0,148,52]
rotor_mixing_plane_faces.append(rotor_mixing_plane_face_to_match.to_dict())  

# Rotor body
block_id = 8; indices = np.array([0,0,0,96,148,0],dtype=int)
rotor_face_to_match,indx = find_face(blocks,block_id, indices,outer_faces)
rotor_faces, outer_faces = find_connected_face(blocks,rotor_face_to_match, outer_faces)
rotor_faces.append(rotor_face_to_match.to_dict())

# Rotor Hub
block_id = 1; indices = np.array([0,0,0,60,0,76],dtype=int)
rotor_hub_to_match,indx = find_face(blocks,block_id, indices,outer_faces)
rotor_hub, outer_faces = find_connected_face(blocks,rotor_hub_to_match, outer_faces)
rotor_hub.append(rotor_hub_to_match.to_dict())

block_id = 7; indices = np.array([0,0,0,60,0,40],dtype=int) 
rotor_hub_to_match,indx = find_face(blocks,block_id, indices,outer_faces)
rotor_hub2, outer_faces = find_connected_face(blocks,rotor_hub_to_match, outer_faces)
rotor_hub2.append(rotor_hub_to_match.to_dict())
rotor_hub.extend(rotor_hub2)

block_id = 10; indices = np.array([0,0,0,96,0,48],dtype=int) 
rotor_hub_to_match,indx = find_face(blocks,block_id, indices,outer_faces)
rotor_hub3, outer_faces = find_connected_face(blocks,rotor_hub_to_match, outer_faces)
rotor_hub3.append(rotor_hub_to_match.to_dict())
rotor_hub.extend(rotor_hub3)

block_id = 14; indices = np.array([0,0,0,40,0,116],dtype=int) 
rotor_hub_to_match,indx = find_face(blocks,block_id, indices,outer_faces)
rotor_hub4, outer_faces = find_connected_face(blocks,rotor_hub_to_match, outer_faces)
rotor_hub4.append(rotor_hub_to_match.to_dict())
rotor_hub.extend(rotor_hub4)

# block_id = 13; indices = np.array([0,0,0,8,0,48],dtype=int) 
# rotor_hub_to_match,indx = find_face(blocks,block_id, indices,outer_faces)
# rotor_hub5, outer_faces = find_connected_face(blocks,rotor_hub_to_match, outer_faces)
# rotor_hub5.append(rotor_hub_to_match.to_dict())
# rotor_hub.extend(rotor_hub5)

# Rotor Shroud
block_id = 1; indices = np.array([0,148,0,60,148,76],dtype=int)
rotor_shroud_to_match,indx = find_face(blocks,block_id, indices,outer_faces)
rotor_shroud, outer_faces = find_connected_face(blocks,rotor_shroud_to_match, outer_faces)
rotor_shroud.append(rotor_shroud_to_match.to_dict())

block_id = 12; indices = np.array([0,148,0,96,148,48],dtype=int) 
rotor_shroud_to_match,indx = find_face(blocks,block_id,indices,outer_faces)
rotor_shroud2, outer_faces = find_connected_face(blocks, rotor_shroud_to_match, outer_faces)
rotor_shroud2.append(rotor_shroud_to_match.to_dict())
rotor_shroud.extend(rotor_shroud2)

block_id = 7; indices = np.array([0,148,0,60,148,40],dtype=int) 
rotor_shroud_to_match,indx = find_face(blocks,block_id, indices,outer_faces)
rotor_shroud3, outer_faces = find_connected_face(blocks,rotor_shroud_to_match, outer_faces)
rotor_shroud3.append(rotor_hub_to_match.to_dict())
rotor_shroud.extend(rotor_shroud3)

block_id = 14; indices = np.array([0,148,0,40,148,116],dtype=int) 
rotor_shroud_to_match,indx = find_face(blocks,block_id, indices,outer_faces)
rotor_shroud4, outer_faces = find_connected_face(blocks,rotor_shroud_to_match, outer_faces)
rotor_shroud4.append(rotor_shroud_to_match.to_dict())
rotor_shroud.extend(rotor_shroud4)


data['outer_faces'] = outer_faces
data['mixing_plane'] = rotor_mixing_plane_faces
data['rotor_body'] = rotor_faces
data['rotor_hub'] = rotor_hub
data['rotor_shroud'] = rotor_shroud

with open('rotor_split_connectivity_final.pickle','wb') as f:
    pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

# Outlet
block_id = 17; indices = np.array([28,0,0,28,148,116],dtype=int)
rotor_outlet,indx = find_face(blocks,block_id, indices,outer_faces)
outer_faces.pop(indx)

# Add matching near leading edge
face_match_1,indx = find_face(blocks,10, np.array([12,0,48,52,148,48], dtype=int),outer_faces)
outer_faces.pop(indx)
face_match_2,indx = find_face(blocks,6, np.array([60,0,0,60,148,40], dtype=int),outer_faces)
outer_faces.pop(indx)
data['face_matches'].append({'block1':face_match_1.to_dict(),
                            'block2':face_match_2.to_dict()})

data['outlet'] = [rotor_outlet.to_dict()]
data['outer_faces'] = outer_faces

with open('rotor_split_connectivity_final.pickle','wb') as f:
    pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)
    


# Putting it all together
If you re-run the `pv_stator_plot.py` and `pv_rotor_plot.py` and change the link to the latest pickle file, you'll find that there's no `outer_faces` anymore. This is what we want. 

The next step is to combine the stator and rotor final connectivity pickle file into a single connectivity file. When we do this, what changes is the `block_index` for the rotor connectivity. 

In [ ]:
'''
    Combines the connectivty and constructs the glennht boundary conditions file 
'''
import sys
sys.path.insert(0,'C:\GitHub\Plot3D_utilities\python')
from typing import Dict, List
import pickle, json, math, copy
from glennht_library import CheckDictionary, modify_bc_template_file, print_connectivity, sutherland
from plot3d import Block,read_plot3D, write_plot3D
from thermo import Mixture


def read_and_shift_block_index(filename:str, block_starting:int=0,blade_name:str='stator'):
    def advance_block_index1(data:List[Dict[str,int]]):
        for i in range(len(data)):
            data[i]['block_index'] += block_starting
        return data

    def advance_block_index2(matches:List[Dict[str,int]]):
        for i in range(len(matches)):
            matches[i]['block1']['block_index']+=block_starting
        return matches

    with open(filename,'rb') as f:
        data = pickle.load(f)
        matches = data['face_matches']
        advance_block_index2(matches)
        
        shroud = CheckDictionary(data,f'{blade_name}_shroud')
        advance_block_index1(shroud)

        hub = CheckDictionary(data,f'{blade_name}_hub')
        advance_block_index1(hub)

        body = CheckDictionary(data,f'{blade_name}_body')
        advance_block_index1(body)

        outer_faces = CheckDictionary(data,'outer_faces')
        advance_block_index1(outer_faces)

        periodic = CheckDictionary(data,'periodic_faces')
        advance_block_index2(periodic)

        mixing_plane = CheckDictionary(data,'mixing_plane')
        advance_block_index1(mixing_plane)
        
        inlet = CheckDictionary(data,'inlet')
        advance_block_index1(inlet)
        
        outlet = CheckDictionary(data,'outlet')
        advance_block_index1(outlet)
        
    return matches,periodic,shroud,hub,body,outer_faces,mixing_plane,inlet,outlet

'''
    We shouldn't have any outerfaces in this step. All outerfaces should either be a hub, shroud, or body.
'''
stator_blocks = read_plot3D('stator_split.xyz',binary=True)
rotor_blocks = read_plot3D('rotor_split.xyz',binary=True)

# Read in all the connectivity 
stator_matches,stator_periodic,stator_shroud,stator_hub,stator_body,_,stator_mixing_plane,stator_inlet,_ = read_and_shift_block_index('stator_split_connectivity_final.pickle')
rotor_matches,rotor_periodic,rotor_shroud,rotor_hub,rotor_body,_,rotor_mixing_plane,_,rotor_outlet = read_and_shift_block_index('rotor_split_connectivity_final.pickle', block_starting=len(stator_blocks),blade_name='rotor')

# Lets combine these connectivities
stator_matches.extend(rotor_matches)
stator_matches.extend(stator_periodic)
stator_matches.extend(rotor_periodic)

faces_and_types = {
                    0:{
                        "faces":stator_inlet,
                        "type":"inlet"
                    },
                    1:{
                        "faces":rotor_outlet,
                        "type":"outlet"
                    },
                    2:{
                        "faces":stator_shroud,
                        "type":"wall"
                    },
                    3:{
                        "faces":stator_hub,
                        "type":"wall"
                    },
                    4:{
                        "faces":stator_body,
                        "type":"wall"
                    },
                    5:{
                        "faces":rotor_shroud,
                        "type":"wall"
                    },
                    6:{
                        "faces":rotor_hub,
                        "type":"wall"
                    },
                    7:{
                        "faces":rotor_body,
                        "type":"wall"
                    },
                    8:{
                        "faces":stator_mixing_plane,
                        "type":"gif"
                    },
                    9:{
                        "faces":rotor_mixing_plane,
                        "type":"gif"
                    }
                }

gifs = [{
        'surface_pairs': [8,9],
        'gif_type': -2
        }]

zones = {'fluid':[], 'solid':[]} # which block indicies are fluid or solid 

# All blocks are fluid
stator_blocks.extend(rotor_blocks)
for i in range(len(stator_blocks)):
    zones['fluid'].append(1)

print_connectivity('stage_connectivity.ght_conn',stator_matches,faces_and_types, gifs, zones)

# Scale mesh in inches to meters
[b.scale(0.0254) for b in stator_blocks] # convert to meters
write_plot3D('StageMesh_metric.xyz',stator_blocks,binary=True)
# Normalize the boundary conditions
with open('settings.json','r') as f:
    data = json.load(f)
    bcs = data['boundary_conditions']
    # Perform calculations
    air = Mixture('air', T=bcs['T0_Inlet'], P=bcs['P0_Inlet'])
    gamma = air.Cp/air.Cvg
    mu,k = sutherland(bcs['T0_Inlet'])
    Pr = air.Cp * mu / k
    Ts = 1/(1+(gamma-1)/2 * bcs['Mach_Inlet']*bcs['Mach_Inlet']) * bcs['T0_Inlet']
    V_rel = bcs['Mach_Inlet'] * math.sqrt(gamma*287*Ts)

    # Non dimensionalize
    bcs_non_dim = copy.deepcopy(bcs)
    bcs_non_dim['Ps_outlet'] /= bcs['P0_Inlet']
    bcs_non_dim['Pref'] = bcs['P0_Inlet']
    bcs_non_dim['Tref'] = bcs['T0_Inlet']
    bcs_non_dim['refPr'] = Pr
    bcs_non_dim['k'] = k
    bcs_non_dim['mu'] = mu
    bcs_non_dim['rho'] = air.rho

    bcs_non_dim['gamma'] = gamma
    bcs_non_dim['Cp'] = air.Cp
    bcs_non_dim['V_rel'] = V_rel
    bcs_non_dim['Re'] = air.rho * V_rel * bcs_non_dim['refLen'] / mu
    
    modify_bc_template_file('boundary_conditions_template.bcs', bcs_non_dim) # <- boundary conditions file already have defined surfaces for inlet, stator hub, stator shroud, rotor hub, rotor shroud, mixing plane, outlet. 

# Final Thoughts
The important outputs of this tutorial is the pickle file describing the connectivity which can be converted into any connectivity file you want. You can also write code to output the surfaces to a boundary conditions file if needed. An example of how to output to GlennHT Boundary (.bcs) boundary conditions file is presented. 